---
## Imports
---

In [39]:
import pandas as pd
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st
import time
from datetime import datetime
import smtplib, ssl
load_dotenv()

True

---
## Server Funcitonality
---

In [5]:
# Pull Log
# Evaluate traders
	# Trading period
	# Trading metrics
	# Send Eval Report (open source/show functions)
	
# Distribute Subscriptions


---
## Client Functionality
---

### Trader Functionality


In [6]:
from Libraries.web3_contract import load_web3
contract,address,w3 = load_web3(".\Contracts\ABI\Contract_abi.json")
address

'0x12e8e73231Dd04547C1c50e6f0B0E9a6cBfA4d08'

In [7]:
# For interacting with the market (Opening/Closing Trades) utilize this method
# Easy to keep things consistent and trade out trading platforms
from datetime import datetime
from Libraries.TradingPlatforms import trade_platforms, init_TradingPlatform
test_platform = init_TradingPlatform(trade_platforms['alpaca'],contract)
test_platform.__hello__()

'This is a Alpaca trading platform.'

In [8]:
TraderAddress = address
TraderID= 1
Open= 1
Symbol= "BTC"
Size= 1
EntryPrice= test_platform.platform.trade_api #TODO get price
EntryTime= datetime.now()
ExpirationTimeStamp= "NOT USED"
Strike= "NOT USED"
IsCall= "NOT USED"

test_platform.openTrade(TraderAddress,TraderID,Open,Symbol,Size,EntryPrice,EntryTime,ExpirationTimeStamp,Strike,IsCall)

AttributeError: 'str' object has no attribute 'trade_api'

### Viewer Functionality


In [ ]:
# Subscribe

---
## Client Interface
---


### Trader Interface


In [1]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("APCA_API_KEY_ID")
API_SECRET = os.getenv("APCA_API_SECRET_KEY")
BASE_URL = os.getenv("APCA_API_BASE_URL")
alpaca_trader = tradeapi.REST(API_KEY, API_SECRET, BASE_URL)

In [2]:
alpaca_trader.get_bars("AAPL",'1Min',limit=1).df['close'][0]

160.44

In [14]:
alpaca_trader.list_orders()[0].status in ["accepted","accepted_for_bidding","calculated","done_for_day","filled","new","partially_filled"]

True

In [30]:
########## SideBar ##########
# list(trade_platforms.values())
# Select Trade Type Dropdown
# Start Trading Button

########## Tabs ##########
# Enter Trade
# Close Trade (from radio or dropdown)

EntryPrice=2.50
EntryTime=int(time.mktime(datetime.now().timetuple()))
theList = [EntryPrice,EntryTime]
list2 = ["None", "N/A", "None"]

In [37]:
def list_to_string(inputList):
    outString = ""
    count = 0
    limit = len(inputList)-1
    for item in inputList:
        if type(item) == datetime:
            outString += item.strftime("%m/%d/%Y, %H:%M:%S")
        else:
            outString += str(item)
        outString
        if (count < limit):
            outString += ", "
        count+=1
    return outString
theString = list_to_string(theList)
theString

'2.5, 1659750918'

In [ ]:
def string_to_list_end_dateTime(inputString):
    splitList = inputString.split(", ")
    outList = []
    for item in splitList:
        if (item != splitList[len(splitList)-1]):
            outList.append(float(item))
        else:
            outList.append(pd.to_datetime(item, unit='s'))
    return outList
otherEnd = string_to_list_end_dateTime(theString)
otherEnd

[2.5, Timestamp('2022-07-30 18:13:38')]

In [ ]:
def input_date_string(date):
    return int(time.mktime(pd.to_datetime(date).timetuple()))
input_date_string(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

1659399117

### Viewer Interface

In [ ]:
# Enter Subscription
# View Subscriptions
	# Filter from logger and grader


In [46]:
sub_df = pd.read_csv("Libraries/sub.csv")
sub_df

,Unnamed: 0,trader_log_address,subscriber_address,email_address,trade_id
0,0,0xAaaC3f8bbEA924c973251A4eb99Ae3497069b904,0x799C8598540Ec3F0C7DC4e9fcba676a92D564a81,jhg55555@gmail.com,1


In [51]:
new_trade_event=[{'args': {'TraderId': 1, 'inputTraderAddress': '0x2301A3e6E59d985385ab0D0f85be56ccD8237abA', 'tradeNum': 3, 'inputOpen': True, 'inputSymbol': 'AAPL', 'inputSize': '10.0', 'inputEntryPriceEntryTime': '165.99, 08/05/2022, 22:07:09', 'optionsData': 'None, N/A, None'}, 'event': 'newTrade', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0xcdcebc452dd4ca9f533d5b17464d529eccf6aa4e0eb1d8e299a52cdb3534ce20', 'address': '0xAaaC3f8bbEA924c973251A4eb99Ae3497069b904', 'blockHash': '0x3d42652d274246b05490f8d203b255ceb6982d12f9da8ad93d25171738bd2613', 'blockNumber': 10}]
# new_close_trade_event=

In [52]:
def email_out(message,MailingList):
    print(MailingList)
    print(message)
    if (len(MailingList) != 0):
        port = 465  # For SSL
        smtp_server = "smtp.gmail.com"
        sender_email = "fivetradetiger@gmail.com"
        password = os.getenv("EMAIL_PASSWORD")
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
            print(server.login(sender_email, password))
            server.sendmail(sender_email,MailingList, message) 

def new_trade_email(new_trade_event):
    MailingList = sub_df[sub_df["trade_id"]== new_trade_event[0]['args']['TraderId']]["email_address"]
    message = f"""Subject: Your new open trade

                Your subscribed trader, {new_trade_event[0]['args']['TraderId']}, made the following trade:
                Open {new_trade_event[0]['args']['inputSize']} share(s) of {new_trade_event[0]['args']['inputSymbol']} at price ${new_trade_event[0]['args']['inputEntryPriceEntryTime']}, options data is {new_trade_event[0]['args']['optionsData']}.
                
                """
    email_out(message,MailingList)

def new_close_trade_email(new_close_trade_event):
    MailingList = sub_df[sub_df["trade_id"]== new_close_trade_event[0]['args']['TraderId']]["email_address"]
    message = f"""Subject: Your next close trade

                Your subscribed trader, {new_close_trade_event[0]['args']['TraderId']}, closed the following trade:
                Close {new_close_trade_event[0]['args']['tradeNum']} at price ${new_close_trade_event[0]['args']['exitPriceExitTime']}.
                
                """
    email_out(message, MailingList)


In [53]:
new_trade_email(new_trade_event)

0    jhg55555@gmail.com
Name: email_address, dtype: object
Subject: Your new open trade

                Your subscribed trader, 1, made the following trade:
                Open 10.0 share(s) of AAPL at price $165.99, 08/05/2022, 22:07:09, options data is None, N/A, None.
                
                
(235, b'2.7.0 Accepted')


In [ ]:
new_close_trade_email(new_close_trade_event)